# Geometric Gauges: Plane and Simple

In [1]:
%pip install -q kingdon anywidget==0.9.13

Note: you may need to restart the kernel to use updated packages.


In [2]:
from kingdon import Algebra
import numpy as np
import itertools
import timeit
import ipywidgets as ipy

alg = Algebra(2, 0, 1)
locals().update(alg.blades)

options = dict(
    lineWidth=4,
    pointRadius=2.5,
    fontSize=3,
    scale=1,
)
animated_options = dict(animate=True, **options)
clrs = [0xff9900, 0xfed290, 0x009977]

In [3]:
# Construct points and lines from coefficients
point = lambda x, y: (e0 + x*e1 + y*e2).dual()
line = lambda a, b, c: a*e1 + b*e2 + c*e0

# The formulas for projection and rejection are the same for all elements of geometry in PGA
project = lambda a, b: (a | b) / b
reject = lambda a, b: (a | b)

In [4]:
# https://enkimute.github.io/ganja.js/examples/coffeeshop.html#XOWMGemyJ

# Initiate a shape
coords = np.ones((3, 5))
coords[1:] = np.random.uniform(0.5, 1.5, size=(2, 5))
points1 = alg.vector(coords).dual()
points2 = points1.map(lambda v: np.roll(v, 1, axis=-1))
shape = list(zip(points1, points2))

# Reflections
origin = alg.blades.e0.dual()
p1 = alg.vector(e0=1, e1=0, e2=0).dual()
p2 = alg.vector(e0=1, e1=0, e2=1).dual()
p3 = alg.vector(e0=1, e1=1, e2=0.3).dual()
u = lambda: (p1 & p2).normalized()
L1 = alg.vector(e1=1).normalized()
L2 = alg.vector(e1=1, e2=0.5).normalized()
L3 = alg.vector(e1=1, e0=0.5).normalized()

## Introduction



# About Reflections

In $d$ dimensions, a single reflection inverts the entire space except for the $d-1$ dimensional subspace in which we reflect.

In [5]:
def refl_graph_func():
    t = timeit.default_timer() / 25
    # Create the reflected shape and the lines between them
    R = np.cos(t) + origin*np.sin(t)
    L1p = R >> L1
    _rpoints1 = L1p >> points1
    _rpoints2 = L1p >> points2
    rshape = zip(_rpoints1, _rpoints2)
    rlines = zip(_rpoints1, points1)
    
    return [
        L1p,
        clrs[0],
        *shape, 
        clrs[2],
        *rshape,
        '<G stroke-width="0.002">',clrs[0],
        *rlines,
        '</G>',
    ]

refl_graph = alg.graph(
    refl_graph_func, **animated_options
)
refl_graph
# point_graph = alg.graph(
#     point_graph_func,
#     **options
# )

# grid = ipy.GridspecLayout(1, 2)
# grid[0, 0] = refl_graph
# grid[0, 1] = point_graph
# grid

GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e01', 'e02', 'e12', 'e012'], ['e0', '0', 'e01', 'e02', '0', '0', …

## Bireflections

Two reflections can form a translation, a rotation, or a boost (in hyperbolic space). These are all examples of **bireflections**.

In [6]:
def bi_refl_graph_func():
    # Reflect the points once and then once more.
    rpoints1 = u >> points1
    rrpoints1 = L1 >> rpoints1
    rpoints2 = u >> points2
    rrpoints2 = L1 >> rpoints2
    
    # Create the reflected shape and the lines between them
    rshape = zip(rpoints1, rpoints2)
    rrshape = zip(rrpoints1, rrpoints2)
    rlines = zip(rpoints1, points1)
    rrlines = zip(rpoints1, rrpoints1)
    
    return [
        p1, p2,
        u,
        L1,
        clrs[0],
        *shape, 
        '<G stroke-dasharray="0.02 0.02">',clrs[1],
        *rshape,
        '</G>',
        '<G stroke-width="0.002">',clrs[0],
        *rlines,
        *rrlines,
        '</G>',
        clrs[2], 
        *rrshape,
    ]

def point_graph_func():
    # Create the reflected shape and the lines between them
    point_reflected_points1 = p1 >> points1
    point_reflected_points2 = p1 >> points2
    rshape = zip(point_reflected_points1, point_reflected_points2)
    rlines = zip(point_reflected_points1, points1)
    
    return [
        p1,
        clrs[0],
        *shape, 
        clrs[2],
        *rshape,
        '<G stroke-width="0.002">',clrs[0],
        *rlines,
        '</G>',
    ]

In [7]:
bi_refl_graph = alg.graph(
    bi_refl_graph_func,
    **options
)

# point_graph = alg.graph(
#     point_graph_func,
#     **options
# )

bi_refl_graph
# grid = ipy.GridspecLayout(1, 2)
# grid[0, 0] = bi_refl_graph
# grid[0, 1] = point_graph
# grid

GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e01', 'e02', 'e12', 'e012'], ['e0', '0', 'e01', 'e02', '0', '0', …

- Identity is the special case of two identical reflections $\implies$ Reflections are *involutary*.
  > Performing the same reflection twice is the same as doing nothing
- Special case are two orthogonal reflections, which is identical to a point reflection.
  > A point(-reflection) **is identical to** two orthogonal line(-reflection)s.

## Bireflections have Geometric Gauges

Given the input and output shape, you could reconstruct the bireflection between them. However, you could never know which specific reflections were used to make that bireflection! This means bireflections have a **gauge** degree of freedom!

In [8]:
def graph_func(L1, L2):
    intersection = L1 ^ L2
    if not intersection**2:
        R = lambda t: 1 + intersection*np.sin(5*t)
    else:
        R = lambda t: np.cos(t) + intersection*np.sin(t)
        
    def _graph_func():
        t = timeit.default_timer() / 25
        L1p = L1
        L2p = R(t) >> L2
        # Reflect the points once and then once more.
        rpoints1 = L1p >> points1
        rrpoints1 = L2p >> rpoints1
        rpoints2 = L1p >> points2
        rrpoints2 = L2p >> rpoints2
        # Create the reflected shape and the lines between them
        rshape = zip(rpoints1, rpoints2)
        rrshape = zip(rrpoints1, rrpoints2)
        rlines = zip(rpoints1, points1)
        rrlines = zip(rpoints1, rrpoints1)
        
        return [
            L1p,
            L2p,
            0xff9900,
            *shape, 
            '<G stroke-dasharray="0.02 0.02">',0xfed290,
            *rshape,
            '</G>',
            '<G stroke-width="0.002">',0xff9900,
            *rlines,
            *rrlines,
            '</G>',
            0x009977, 
            *rrshape,
        ]
    return _graph_func
    
rot_graph = alg.graph(
    graph_func(L1, L2),
    **animated_options
)

trans_graph = alg.graph(
    graph_func(L1, L3),
    **animated_options
)

grid = ipy.GridspecLayout(1, 2)
grid[0, 0] = rot_graph
grid[0, 1] = trans_graph
grid

GridspecLayout(children=(GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e01', 'e02', 'e12', 'e012'], ['e0', '0',…

## Cartan-Dieudonné theorem

**Geometric Gauges** are the geometric mechanism behind the famous *Cartan-Dieudonné theorem*:
> Every orthogonal transformation in an $n$-dimensional embedding space is composed from at most $n$ reflections. 

<iframe src="https://enkimute.github.io/ganja.js/examples/coffeeshop.html#Mqs0ezNVP&fullscreen" width="100%" height="100%" frameBorder="0"></iframe>

# Geometric Gauges explain all Products!

## Sandwich


## First Principle

## Invariant Decomposition

$$ b_i = \frac{\lambda_i B + \tfrac{1}{3!} B \wedge B \wedge B}{\lambda_i + \tfrac{1}{2} B \wedge B} $$

## Conclusion

End by qouting Charles like in the GSG paper?

<iframe src="https://enkimute.github.io/ganja.js/examples/coffeeshop.html#XOWMGemyJ&fullscreen" width="100%" height="100%" frameBorder="0"></iframe>